# Function Calling with LLMs

This notebook demonstrates how to use LLMs to call external functions/tools.

We'll cover:
1. Basic function binding and invocation
2. Creating a tool-calling agent
3. Chaining multiple tool calls

Function calling allows LLMs to:
- Execute code
- Access external APIs
- Perform complex calculations
- Integrate with other systems

In [ ]:
from dotenv import load_dotenv
from rich import print

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)

In [ ]:
from genai_tk.core.llm_factory import get_llm
from genai_tk.core.prompts import def_prompt
from langchain_core.tools import tool

## Basic Function Calling

Here we demonstrate the core function calling workflow:
1. Define a simple addition function
2. Bind it to the LLM
3. Ask the LLM to use the function

The LLM will:
- Recognize when to use the function
- Generate the correct parameters
- Return the function call details

In [ ]:
llm = get_llm("gpt_4o_azure")
llm = get_llm()
messages = []


@tool
def add(a: int, b: int) -> int:
    """Addition 2 integer numbers a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add], tool_choice="any")  # tool_choice="any"
prompt = def_prompt(
    "Use the provided functions  to compute what is 45 + 12; execute the function. Return 'I don't know' if there are not relevant function  "
)

chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

print(ai_msg)

In [ ]:
print(llm_with_tools)

## Function Execution

After the LLM returns a function call:
1. We execute the function locally
2. Pass the result back to the LLM
3. Let the LLM process the final response

This creates a complete function calling loop.

In [ ]:
# Step 2 : Execute the function, and send result to LLM

## Tool-Calling Agent

For more complex scenarios, we can create an agent that:
- Manages multiple tools
- Handles tool selection
- Processes tool results
- Maintains conversation state

Here we create a math assistant agent with:
- Addition
- Exponentiation capabilities

In [ ]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

llm = get_llm("deepseek_v31_openrouter")


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)."""
    print("exponentiate")
    return math.pow(x, y)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant. Please use the provided tool"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
# r = agent_executor.invoke({"input": "What is the temperature in Toulouse today ? "})
print(r)